# 本地API测试

In [2]:
import requests

r = requests.get('http://127.0.0.1:5000/v1/models')
models = [
    model['id']
    for model in r.json()['data']
]

model_name = models[0] # select first
models

['/models/codellama-7b-instruct.Q5_K_M.gguf']

In [4]:
import openai

openai.api_base = "http://127.0.0.1:5000/v1"

# create a chat completion
completion = openai.ChatCompletion.create(
  model=model_name,
  messages=[{"role": "user", "content": "你好，请自我介绍并说明你能给我提供的帮助"}]
)
# print the completion
print(completion.choices[0].message.content)

你好！我是Assistant。我能够回答


In [5]:
completion = openai.ChatCompletion.create(
  model=model_name,
  messages=[
      {"role": "user", "content": """您是个拥有丰富经验的AI助教，帮助或指引学生解决学习中遇到的问题。
约束：对于已知问题的解答，需要通过 `从xxx的信息可以得出...` 说明其出处，其中xxx是实际存在的文章段落；对于未知或不确定的知识，请说明你不具备这部分知识，并提供一些相关主题的参考建议。
我的第一个问题，请帮忙解释下下面代码：asyncio.wait()"""},
  ]
)

print(completion.choices[0].message.content)

您需要了解的是 `asyncio.wait()`


# zhipuai bridge

In [3]:
import json
import aiohttp
import asyncio
import zhipuai
from typing import AsyncGenerator, Tuple, Union
from zhipuai.model_api.api import ModelAPI, InvokeType
from zhipuai.utils import jwt_token

zhipuai.api_key = "65ee3a27e9a80f04abad9af2a99de2e8.sinGhMdqrHOblWRZ"

class DictObject(dict):
    "A dict base object like OpenAIObject"
    # https://github.com/openai/openai-python/blob/1be14ee34a0f8e42d3f9aa5451aa4cb161f1781f/openai/openai_object.py#L11

    def __init__(
        self,
        id=None,
        **params,
    ):
        super(DictObject, self).__init__()

        self._params = params

        if id:
            self["id"] = id

    def __setattr__(self, k, v):
        if k[0] == "_" or k in self.__dict__:
            return super(DictObject, self).__setattr__(k, v)

        self[k] = v
        return None

    def __getattr__(self, k):
        if k[0] == "_":
            raise AttributeError(k)
        try:
            return self[k]
        except KeyError as err:
            raise AttributeError(*err.args)

    def __delattr__(self, k):
        if k[0] == "_" or k in self.__dict__:
            return super(DictObject, self).__delattr__(k)
        else:
            del self[k]

    def __setitem__(self, k, v):
        if v == "":
            raise ValueError(
                "You cannot set %s to an empty string. "
                "We interpret empty strings as None in requests."
                "You may set %s.%s = None to delete the property" % (k, str(self), k)
            )
        super(DictObject, self).__setitem__(k, v)

    def __delitem__(self, k):
        raise NotImplementedError("del is not supported")

    def __str__(self):
        obj = self.to_dict_recursive()
        return json.dumps(obj, indent=2)

    def to_dict(self):
        return dict(self)

    def to_dict_recursive(self):
        d = dict(self)
        for k, v in d.items():
            if isinstance(v, DictObject):
                d[k] = v.to_dict_recursive()
            elif isinstance(v, list):
                d[k] = [
                    e.to_dict_recursive() if isinstance(e, DictObject) else e
                    for e in v
                ]
        return d

    @classmethod
    def construct_from(cls, values):
        instance = cls()
        instance.refresh_from(values)
        return instance

    def refresh_from(self, values):
        # Wipe old state before setting new.
        self.clear()
        for k, v in values.items():
            super(DictObject, self).__setitem__(
                k, convert_to_dict_object(v)
            )

        self._previous = values

    # This class overrides __setitem__ to throw exceptions on inputs that it
    # doesn't like. This can cause problems when we try to copy an object
    # wholesale because some data that's returned from the API may not be valid
    # if it was set to be set manually. Here we override the class' copy
    # arguments so that we can bypass these possible exceptions on __setitem__.
    def __copy__(self):
        copied = DictObject()

        copied._params = self._params

        for k, v in self.items():
            # Call parent's __setitem__ to avoid checks that we've added in the
            # overridden version that can throw exceptions.
            super(DictObject, copied).__setitem__(k, v)

        return copied

    # This class overrides __setitem__ to throw exceptions on inputs that it
    # doesn't like. This can cause problems when we try to copy an object
    # wholesale because some data that's returned from the API may not be valid
    # if it was set to be set manually. Here we override the class' copy
    # arguments so that we can bypass these possible exceptions on __setitem__.
    def __deepcopy__(self, memo):
        copied = self.__copy__()
        memo[id(self)] = copied

        for k, v in self.items():
            # Call parent's __setitem__ to avoid checks that we've added in the
            # overridden version that can throw exceptions.
            super(OpenAIObject, copied).__setitem__(k, deepcopy(v, memo))

        return copied

def convert_to_dict_object(data):
    if isinstance(data, list):
        return [
            convert_to_dict_object(v)
            for v in data
        ]
    elif isinstance(data, dict) and not isinstance(data, DictObject):
        return DictObject().construct_from(data)
    else:
        return data

class ZPChatCompletion(ModelAPI):
    # OBJECT_NAME = "chat.completions"

    @classmethod
    def create(cls,
        api_key=None,
        api_base=None,
        api_type=None,
        request_id=None,
        api_version=None,
        organization=None,
        **params,
     ):
        """
        Creates a new chat completion for the provided messages and parameters.
        """
        # if api_key: rewrite api_key
        zhipu_params = cls.parse_to_zhipu_request(params)
        response = super().invoke(**zhipu_params)
        return cls.convert_to_openai_response(response)

    @classmethod
    async def acreate(cls,
        api_key=None,
        api_base=None,
        api_type=None,
        request_id=None,
        api_version=None,
        organization=None,
        **params,
    ):
        """
        Creates a new chat completion for the provided messages and parameters.
        """
        zhipu_params = cls.parse_to_zhipu_request(params)
        url = ModelAPI._build_api_url(zhipu_params, InvokeType.SYNC)
        response = await APIRequestor().arequest(
            "post",
            url,
            params=json.dumps(zhipu_params),
            headers={
                "Authorization": jwt_token.generate_token(zhipuai.api_key),
                "Accept": "application/json",
                "Content-Type": "application/json; charset=UTF-8",
            },
        )
        return cls.convert_to_openai_response(await response.json())
    
    @classmethod
    def parse_to_zhipu_request(cls, params):
        if 'messages' not in params:
            raise Exception("messages param not found")
        messages = params.pop('messages')
        zhipu_messages = []
        for msg in messages:
            # {"role": "system|user|assistant", "content": "text"}
            if msg['role'] == 'system':  # not supported by ZhiPuAI, translate to user prompt
                zhipu_messages.append({"role": 'user', "content": msg['content']})
            else:
                zhipu_messages.append(msg)
        params['prompt'] = zhipu_messages
        return params  

    @classmethod
    def convert_to_openai_response(cls, resp):
        data = resp.pop('data')
        request_id = data.pop('request_id')

        choices = data.pop('choices')
        openai_choices = []
        if len(choices) > 0:
            for i, msg in enumerate(choices):
                openai_choices.append(convert_to_dict_object({
                    "message": msg,
                    "finish_reason": "",
                    "index": i,
                }))
            openai_choices[-1]['finish_reason'] = "stop"

        resp_object = convert_to_dict_object(resp)
        resp_object.request_id = request_id
        resp_object.choices = openai_choices
        return resp_object

class APIRequestor:

    async def arequest(
        self,
        method,
        url,
        params=None,
        headers=None,
        proxy=None,
        timeout=None,
        # stream: bool = False,
    ) -> Tuple[Union[DictObject, AsyncGenerator[DictObject, None]], bool, str]:
        request_kwargs = {
            "method": method,
            "url": url,
            "headers": headers,
            "data": params,
            "proxy": proxy,
            "timeout": timeout,
        }

        async with aiohttp.ClientSession() as session:
            try:
                result = await session.request(**request_kwargs)
                # Don't read the whole stream for debug logging unless necessary.
                return result
            except (aiohttp.ServerTimeoutError, asyncio.TimeoutError) as e:
                raise error.Timeout("Request timed out") from e
            except aiohttp.ClientError as e:
                raise error.APIConnectionError("Error communicating with ZhiPuAI") from e

import openai
import importhook

# Setup hook to be called any time the `openai` module is imported and loaded into module cache
@importhook.on_import('openai')
def on_openai_import(socket):
    new_openai = importhook.copy_module(openai)
    setattr(new_openai, 'ChatCompletion', ZPChatCompletion)
    return new_openai

In [2]:
import openai

completion = openai.ChatCompletion.create(
  model="chatglm_pro",
  messages=[{"role": "user", "content": "你好，请自我介绍并说明你能给我提供的帮助"}]
)
print(completion.choices[0].message.content)

" 你好！我是一名人工智能助手，我的主要任务是帮助你解决问题和提供有用的信息。我接受了大量的训练，可以回答各种领域的问题，包括但不限于科学、技术、历史、地理、文化、娱乐等。如果你有任何疑问或者需要帮助，请随时告诉我，我会尽力为你提供解答。"


In [3]:
import openai

acompletion = await openai.ChatCompletion.acreate(
  model="chatglm_pro",
  messages=[{"role": "user", "content": "你好，请自我介绍并说明你能给我提供的帮助"}]
)
print(acompletion.choices[0].message.content)

" 你好！我是 ChatGLM，是清华大学 KEG 实验室和智谱 AI 公司共同训练的语言模型。我的目标是通过回答用户提出的问题来帮助他们解决问题。我可以回答各种问题，例如科学、数学、历史、文化、技术等等。如果有任何问题需要帮助，请随时问我。"


In [4]:
import openai
import semantic_kernel as sk
from loguru import logger
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.ai.chat_request_settings import ChatRequestSettings

settings = ChatRequestSettings(temperature=0.95, top_p=0.7)
chat = OpenAIChatCompletion("chatglm_pro", "noop", log=logger)
resp = await chat.complete_chat_async(messages=[("user", "你好，请自我介绍并说明你能给我提供的帮助")],
                         request_settings=settings, logger=logger)
resp

AttributeError: 

# Semantic Kernel

In [95]:
import semantic_kernel as sk

# import openai
# openai.api_base = "http://127.0.0.1:5001/v1"

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from IPython.display import display, Markdown

In [98]:
kernel = sk.Kernel()
kernel.add_text_completion_service("openai", OpenAIChatCompletion("chatglm_pro", "noop", ""))

sk_prompt = """
您是一位经验丰富的AI商业分析助手。您的主要目标是分辨[问题]涉及的内容，并从[文章]抽取该内容相关的4个客观观点，用于下一步在线搜索的Google查询语句。约束：请考虑该查询是否可以检索到包含业务价值、市场趋势和战略分析相关的问题，以便后续生成全面、富有洞察力、公正并且系统化结构的商业分析报告。

您必须以json代码块的格式输出结果，格式参考:
`["query 1", "query 2", "query 3", "query 4"]`

注意：查询query必须是中文

{{$input}}
"""
summary_function = kernel.create_semantic_function(prompt_template = sk_prompt,
                                                    description="观点抽取",
                                                    max_tokens=200,
                                                    temperature=0.1,
                                                    top_p=0.5)

In [99]:
sk_input = """
主题：AI 与 Web3 的布局机会：投资人眼中的前景和机遇

文章：
上次写完BTC生态的东西，本来该补上NFT，NFTFI的那篇，奈何NFT最近真的不是一般凉，不光二级市场凉，一级市场我好像有俩月都没聊到NFT或是NFTFI相关的项目了，倒是AI项目真的如井喷一般的涌现，所以NFT那篇继续拖着吧，把AI+Web3的结合趋势这篇先提上来。

一 . 先说说AI自身
AI这行业其实本来都要凉凉了，大家知道Near的创始人一龙对吧，这家伙其实以前是做AI的，他是TensorFlow（最流行的机器学习框架）的主要代码贡献者。大家推测他是AI（大模型之前的机器学习）那边看不到啥希望了所以跑来做Web3的。

结果终于去年年底业界迎来了ChatGpt3.5，一下子这行业又活了，因为这次真的可以算质变了，而不是之前那几波的炒作和量变。这不隔了几个月AI创业的浪潮也传递到了我们Web3。硅谷Web2那边则是卷的不行，各种资本Fomo，各种同质化方案开始拼价格战，各种大厂大模型PK……

但是要注意到的是AI经历了半年多的爆发期之后也进入了一个相对瓶颈期，比如Google对与AI的搜索热度断崖式下跌，Chatgpt用户增速大幅放缓，AI Output带有一定的随机性限制了许多落地场景……总而言之，我们离传说中的“AGI - 通用人工智能”还有非常非常远的距离。

目前硅谷创投圈对与AI下一步发展有这么几个判断 ：

1. 没有垂类模型，只有大模型+垂类应用（一会儿说Web3+AI的时候我们会再提到）；

2. 边缘设备比如手机端的数据可能会是个壁垒，基于边缘设备的AI可能也是个机会；

3. Context的长度未来可能引发质变（现在用向量数据库作为AI记忆体，但上下文长度还是不够）。

二. Web3+AI
AI和Web3其实是完全不同的两个领域，AI需要集中的算力+海量数据做训练，非常中心化的东西，Web3则是主打一个去中心化，所以其实不是那么好结合，单奈何叙事上AI改变生产力，区块链改变生产关系这个论点太过深入人心，所以总会有人前仆后继的去寻找那个结合点，近俩月得聊了不下10个AI项目。

在说新的结合赛道之前先说说老的AI+Web3项目，基本都是平台型，以FET和AGIX为代表。怎么说呢，我国内专业做AI的朋友是这么跟我说的 - “以前这些做AI的现在基本都没啥用了，无论Web2还是Web3,很多都是包袱而不是经验。方向和未来就是像OpenAI的这种基于Transformer的大模型，大模型拯救了AI”，你自己品。

所以通用平台型不是他所看好的Web3+AI的模式，我聊的这10多个项目也确实没有这方面，目前看到的基本是如下几个赛道：

1. Bot/Agent/Assistant 模型资产化
2. 算力平台
3. 数据平台
4. 生成式AI
5. Defi交易/审计/风控
6. ZKML

今天主要详细说下1，也就是Bot/Agent/Assitant的资产化这个赛道，这也是聊的最多，同质化最为严重的一个赛道 简单来说，这些项目多是拿OpenAI为底层，配合其他的一些开源/自研的技术手段，比如TTS（Text to Speech）之类，加上特定的数据，FineTune出来一些“某一领域比ChatGPT”更好的机器人。

比如你可以训练出一个教你英语的美女老师，你可以选择她是美国口音还是伦敦腔，她的性格和聊天的方式也可以调整，这样相对于ChatGPT比较机械和官方的回答来讲，你的交互体验会更好一些。圈内前段有个虚拟男友的DAPP，Web3女性向游戏，叫HIM，可以算是这种类型的代表了。

从这个思路出发，你理论上可以有许多个Bot/Agent为你服务，比如你想要做水煮鱼，可能会有专门针对这个领域Fine Tune的Cooking Bot来教你，给的答案相对ChatGPT更加专业，你想出门旅行，同样有旅行小助手Bot给你提供各种出游建议和规划，或是你是项目方，弄一个Discord的客服机器人，帮你回答社区问题。

除了做这种“基于GPT的垂类应用型”Bot，还有基于此的衍生项目，比如Bot算“模型资产化”，有点NFT“小图片资产化”的意味，那现在AI里面流行的Prompt是不是也可以资产化，像是MidJourney不同的Prompt可以生成不同的图片，训练Bot时不同的Prompt也会有不同的效果，所以Promopt自身也具备价值，也可以资产化。

还有像是基于此类Bot进行门户索引，搜索的项目，哪天我们有了成千上万的Bot，怎么找到最合适你的Bot？可能届时就需要一个Web2世界类似Hao123这样的门户，或是Google这样的搜索引擎来帮你“定位”。
""";

summary_result = await kernel.run_async(summary_function, input_str=sk_input)

display(Markdown("### ✨ " + str(summary_result)))

### ✨ Error: (<ErrorCodes.ServiceError: 6>, 'OpenAI service failed to complete the chat', APIConnectionError(message='Error communicating with OpenAI', http_status=None, request_id=None))